# 1. Particle advection

Advection of a regular grid of particles using geostrophic currents derived from altimetry. 

Part of the Master's Thesis of the Master's Degree in Advanced Physics and Applied Mathematics at University of the Balearic Islands (UIB) on Reconstruction of Sea Surface Salinity from observations made by the SMOS satellite in the southwest Atlantic Ocean

by Cristina Martí Solana

_Note: latest versions of OceanParcels are not compatible with this script, **use version 2.4.2**_

Import modules

In [15]:
from parcels import Field, FieldSet, ParticleSet, Variable, JITParticle, AdvectionRK4, plotTrajectoriesFile, ErrorCode
import numpy as np
from pandas import read_csv
import xarray as xr
import os
import math
from datetime import timedelta, datetime
from operator import attrgetter
import matplotlib.pyplot as plt
from matplotlib import dates as mdates
import pickle

Save the outputs in this directory

In [16]:
path_save = "../outputs/sim/"

### Altimetry data

Define altimetry directory and extract dates of delayed-time fields

In [17]:
path_altimetry = "../data/velocity_data/cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1D_1701257133382.nc"
ds = xr.open_dataset(path_altimetry)
dates_altimetry = ds.time
ds.close()

# Define time range of the alimetry dataset 

date_time_obj = datetime.strptime(str(dates_altimetry.values[0])[:10],"%Y-%m-%d")

year = date_time_obj.year
month = date_time_obj.month
day = date_time_obj.day

date_alt_ori = mdates.date2num(datetime(year,month,day,0,0,0)) # First day of the dataset
date_alt_fin = dates_altimetry.values[-1] # Last day of the dataset

### Particle release grid

In [18]:
# spatial domain boundaries
lonmin, lonmax = -70, -35
latmin, latmax = -50, -20

dx = 0.04 # grid resolution

# create grid coordinates
lons,lats = np.meshgrid(np.arange(lonmin,lonmax,dx), np.arange(latmin,latmax,dx))

# longitude and latitude coordinates in a 1D vector
lonf, latf = lons.flatten(), lats.flatten()

Allow particles to leave the domain

In [19]:
def DeleteParticle(particle,fieldset,time):
    particle.delete()

### Define simulation dates

From in-situ data define simulation dates: extract dates from the in-situ mesuraments 

In [20]:
# path of TSG data
path_TSG = "../data/insitu_data/TSG.csv"
ds = read_csv(path_TSG)

dates_TSG = [str(ds["date"][i])[:10] for i in range(len(ds))]

# list of in situ dates without repeated values
dates_sim = [dates_TSG[0]]
for i in range(1,len(dates_TSG)):
    if dates_TSG[i-1] != dates_TSG[i]:
        dates_sim.append(dates_TSG[i])
    
time_lim = ["2016-04-08", "2016-05-12"]

# SMOS SSS L3 maps generated by CATDS CEC LOCEAN (9 days)
path = '../data/SSS_data/SMOS_L3_DEBIAS_LOCEAN_v8/debiasedSSS_09days_v8/*.nc'
ds_SMOS_LOCEAN = xr.open_mfdataset(path)
ds_SMOS_LOCEAN = ds_SMOS_LOCEAN.sel(time=slice(time_lim[0],time_lim[1]))

dates_sim = []
for i in range(len(ds_SMOS_LOCEAN["time"].values)):
    dates_sim.append(str(ds_SMOS_LOCEAN["time"].values[i])[:10])


print("Dates of simulation:", dates_sim)

Dates of simulation: ['2016-04-10', '2016-04-14', '2016-04-18', '2016-04-22', '2016-04-26', '2016-04-30', '2016-05-04', '2016-05-08', '2016-05-12']


### Do simulations for each date on the list

In [21]:
rund = [4] # day simulations

for date in dates_sim:
    date_time_obj = datetime.strptime(date,"%Y-%m-%d") # date of the simulation

    year = date_time_obj.year
    month = date_time_obj.month
    day = date_time_obj.day

    print("Simulation date", year,month,day)

    # Final time of the backward simulation
    date_fin = mdates.date2num(datetime(year,month,day,0,0,0))

    # Define the file of the names of the files containing the zonal (U) and meridional (V) velocities
    filenames = {"U" : path_altimetry,
                 "V" : path_altimetry}
    
    # Then, define a dictionary of the variables (`U` and `V`) and dimensions (`lon`, `lat` and `time`; 
    # note that in this case there is no `depth` because the data is only for the surface of the ocean).

    # Left: name of the variable that Parcels needs
    # Right: name of the variable in the nc file

    variables = {"U" : "ugos",
                 "V" : "vgos"}
    dimensions = {"lat" : "latitude",
                  "lon" : "longitude", 
                  "time" : "time"}
    
    # Finally, read in the fieldset using the "FieldSet.from_netcdf" function with the 
    # above-defined "filenames", "variables" and "dimensions"

    fieldset = FieldSet.from_netcdf(filenames,variables,dimensions)

    # Do all simulations in a loop to test different running times
    # Running particles in backward time is extremely simple: just provide a dt < 0.

    daysf = date_fin - date_alt_ori

    for rd in rund: # number of days of the simulation (run time)
        daysi = daysf # release day is the final date

        timef = np.ones(lonf.shape) * timedelta(days=daysi).total_seconds() # release particle daysi later from 1-Jan-2017

        pset = ParticleSet(fieldset=fieldset, pclass=JITParticle, lon=lonf, lat=latf, time=timef)

        savefilename = "advection" + '%02i'% rd +"days/"+"sim"+mdates.num2date(date_fin).strftime("%Y%m%d")+"_"+'%02i'% rd +'days.zarr' 

        print("Starting simulation for run day ... ", rd)

        pset.execute(AdvectionRK4,
                     runtime=timedelta(days=int(rd)), # total length of the run
                     dt = -timedelta(hours=3),
                     output_file = pset.ParticleFile(name=path_save+savefilename, outputdt=timedelta(hours=12)),
                     recovery = {ErrorCode.ErrorOutOfBounds: DeleteParticle}) # recovery kernel
        
        print("Simulation saved as ... ", savefilename)

    print("")
    print("Simulations finished for this date")
    print("")
print("SIMULATIONS FINISHED :)")



Simulation date 2016 4 10
Starting simulation for run day ...  4


INFO: Compiled ArrayJITParticleAdvectionRK4 ==> /var/folders/1q/fy668b553sz21tyn8_n_991c0000gn/T/parcels-501/libde5c1f1841c7bc96b2ea31b999f68a0c_0.so
INFO: Output files are stored in ../outputs/sim/advection04days/sim20160410_04days.zarr.
100%|██████████| 345600.0/345600.0 [00:01<00:00, 180573.73it/s]


Simulation saved as ...  advection04days/sim20160410_04days.zarr

Simulations finished for this date

Simulation date 2016 4 14
Starting simulation for run day ...  4


INFO: Compiled ArrayJITParticleAdvectionRK4 ==> /var/folders/1q/fy668b553sz21tyn8_n_991c0000gn/T/parcels-501/lib734985a8a5f449625f91dc91aac1b86f_0.so
INFO: Output files are stored in ../outputs/sim/advection04days/sim20160414_04days.zarr.
100%|██████████| 345600.0/345600.0 [00:01<00:00, 183849.85it/s]


Simulation saved as ...  advection04days/sim20160414_04days.zarr

Simulations finished for this date

Simulation date 2016 4 18
Starting simulation for run day ...  4


...

Simulation saved as ...  advection04days/sim20160504_04days.zarr

Simulations finished for this date

Simulation date 2016 5 8
Starting simulation for run day ...  4


INFO: Compiled ArrayJITParticleAdvectionRK4 ==> /var/folders/1q/fy668b553sz21tyn8_n_991c0000gn/T/parcels-501/lib1602605a2abcdf301eb2b13eff9daad5_0.so
INFO: Output files are stored in ../../outputs/sim/advection04days/sim20160508_04days.zarr.
100%|██████████| 345600.0/345600.0 [00:01<00:00, 182865.85it/s]


Simulation saved as ...  advection04days/sim20160508_04days.zarr

Simulations finished for this date

Simulation date 2016 5 12
Starting simulation for run day ...  4


INFO: Compiled ArrayJITParticleAdvectionRK4 ==> /var/folders/1q/fy668b553sz21tyn8_n_991c0000gn/T/parcels-501/libde6f4ee8773cbe541906ba72e5e43e6b_0.so
INFO: Output files are stored in ../../outputs/sim/advection04days/sim20160512_04days.zarr.
100%|██████████| 345600.0/345600.0 [00:01<00:00, 180734.89it/s]

Simulation saved as ...  advection04days/sim20160512_04days.zarr

Simulations finished for this date

SIMULATIONS FINISHED :)
